In [2]:
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler

<div id="creating"></a>

### Creating the Training Dataset

In [3]:
current_seeds_file = open("./Data/2024_tourney_seeds.csv")
current_seeds_df = pd.read_csv(current_seeds_file)

m_past_seeds_file = open("./Data/MNCAATourneySeeds.csv")
m_past_seeds = pd.read_csv(m_past_seeds_file)

m_reg_results_file = open("./Data/MRegularSeasonDetailedResults.csv")
m_reg_results_df = pd.read_csv(m_reg_results_file)

m_tourney_results_file = open("./Data/MNCAATourneyCompactResults.csv")
m_tourney_results_df = pd.read_csv(m_tourney_results_file)

In [4]:
m_trainset_full = pd.DataFrame()
past_yearlist = list(range(2003, 2024))
m_tourney_results_df = m_tourney_results_df.where(m_tourney_results_df["Season"] >= 2003).dropna()

m_trainset_seasoncol = []
m_trainset_team0col = []
m_trainset_team1col = []
m_trainset_scorecol = []

for i in range(0, len(m_tourney_results_df["Season"])):
    m_trainset_seasoncol.append(m_tourney_results_df["Season"].tolist()[i])
    m_trainset_seasoncol.append(m_tourney_results_df["Season"].tolist()[i])
    
    m_trainset_team0col.append(m_tourney_results_df["WTeamID"].tolist()[i])
    m_trainset_team0col.append(m_tourney_results_df["LTeamID"].tolist()[i])

    m_trainset_team1col.append(m_tourney_results_df["LTeamID"].tolist()[i])
    m_trainset_team1col.append(m_tourney_results_df["WTeamID"].tolist()[i])

    m_trainset_scorecol.append(m_tourney_results_df["WScore"].tolist()[i])
    m_trainset_scorecol.append(m_tourney_results_df["LScore"].tolist()[i])

m_trainset_full.insert(0, "Season", m_trainset_seasoncol)
m_trainset_full.insert(1, "Team0", m_trainset_team0col)
m_trainset_full.insert(2, "Team1", m_trainset_team1col)
m_trainset_full.insert(3, "Score", m_trainset_scorecol)

m_trainset_full

,Season,Team0,Team1,Score
0,2003.0,1421.0,1411.0,92.0
1,2003.0,1411.0,1421.0,84.0
2,2003.0,1112.0,1436.0,80.0
3,2003.0,1436.0,1112.0,51.0
4,2003.0,1113.0,1272.0,84.0
...,...,...,...,...
2625,2023.0,1274.0,1163.0,59.0
2626,2023.0,1361.0,1194.0,72.0
2627,2023.0,1194.0,1361.0,71.0
2628,2023.0,1163.0,1361.0,76.0


In [5]:
m_predset_full = pd.DataFrame()
m_current_seeds_df = current_seeds_df.where(current_seeds_df["Tournament"] == "M").dropna()
m_current_seeds_teamdict = {i : int() for i in current_seeds_df["Seed"].tolist()}

m_predset_seasoncol = []
m_predset_seedcol = []
m_predset_team0col = []
m_predset_team1col = []

m_current_seeds_w = []
m_current_seeds_w_top = []
m_current_seeds_w_bottom = []

m_current_seeds_x = []
m_current_seeds_x_top = []
m_current_seeds_x_bottom = []

m_current_seeds_y = []
m_current_seeds_y_top = []
m_current_seeds_y_bottom = []

m_current_seeds_z = []
m_current_seeds_z_top = []
m_current_seeds_z_bottom = []

for seed in m_current_seeds_df["Seed"].tolist():
    m_current_seeds_teamdict[seed] = m_current_seeds_df["TeamID"].where(m_current_seeds_df["Seed"] == seed).dropna().tolist()[0]

for i in range(0, len(m_current_seeds_df["Seed"])):
    m_predset_seasoncol.append(2024)

    if "W" in m_current_seeds_df["Seed"].tolist()[i]:
        m_current_seeds_w.append(m_current_seeds_df["Seed"].tolist()[i])
    
    elif "X" in m_current_seeds_df["Seed"].tolist()[i]:
        m_current_seeds_x.append(m_current_seeds_df["Seed"].tolist()[i])

    elif "Y" in m_current_seeds_df["Seed"].tolist()[i]:
        m_current_seeds_y.append(m_current_seeds_df["Seed"].tolist()[i])

    elif "Z" in m_current_seeds_df["Seed"].tolist()[i]:
        m_current_seeds_z.append(m_current_seeds_df["Seed"].tolist()[i])

m_current_seeds_w_top = m_current_seeds_w[:8]
m_current_seeds_w_bottom = m_current_seeds_w[8:][::-1]
m_current_seeds_x_top = m_current_seeds_x[:8]
m_current_seeds_x_bottom = m_current_seeds_x[8:][::-1]
m_current_seeds_y_top = m_current_seeds_y[:8]
m_current_seeds_y_bottom = m_current_seeds_y[8:][::-1]
m_current_seeds_z_top = m_current_seeds_z[:8]
m_current_seeds_z_bottom = m_current_seeds_z[8:][::-1]

m_current_seeds_w = []
m_current_seeds_x = []
m_current_seeds_y = []
m_current_seeds_z = []

for i in range(0, len(m_current_seeds_w_top)):
    m_current_seeds_w.append(m_current_seeds_w_top[i])
    m_current_seeds_w.append(m_current_seeds_w_bottom[i])
    m_current_seeds_x.append(m_current_seeds_x_top[i])
    m_current_seeds_x.append(m_current_seeds_x_bottom[i])
    m_current_seeds_y.append(m_current_seeds_y_top[i])
    m_current_seeds_y.append(m_current_seeds_y_bottom[i])
    m_current_seeds_z.append(m_current_seeds_z_top[i])
    m_current_seeds_z.append(m_current_seeds_z_bottom[i])

m_current_seeds_bracket = m_current_seeds_w + m_current_seeds_x + m_current_seeds_y + m_current_seeds_z

m_current_seeds_bracket_team0 = []
for seed in m_current_seeds_bracket:
    m_current_seeds_bracket_team0.append(m_current_seeds_teamdict[seed])

m_current_seeds_bracket_team1 = []
for i in range(0, (len(m_current_seeds_bracket_team0))):
    if i%2 == 0:
        m_current_seeds_bracket_team1.append(m_current_seeds_bracket_team0[i])
    else:
        m_current_seeds_bracket_team1.insert(i-1, m_current_seeds_bracket_team0[i])

m_predset_full.insert(0, "Season", m_predset_seasoncol)
m_predset_full.insert(1, "Seed", m_current_seeds_bracket)
m_predset_full.insert(2, "Team0", m_current_seeds_bracket_team0)
m_predset_full.insert(3, "Team1", m_current_seeds_bracket_team1)

In [6]:
m_team0_dict = {l : list() for l in list(range(2003, 2025))}
m_team1_dict = {l : list() for l in list(range(2003, 2025))}

for year in m_team0_dict.keys():
    if year == 2024:
        m_team0_dict[year] = m_predset_full["Team0"].tolist()
        m_team1_dict[year] = m_predset_full["Team1"].tolist()
    else:
        m_team0_dict[year] = m_trainset_full["Team0"].where(m_trainset_full["Season"] == year).dropna().tolist()
        m_team1_dict[year] = m_trainset_full["Team1"].where(m_trainset_full["Season"] == year).dropna().tolist()

<div id="transforming"></a>

### Transforming Data into Metrics

In [7]:
# Metric Dictionary
yearlist = list(range(2003, 2025))
metriclist = ["AVG_PPG", "FG_ATT", "FG_PCT", "3PT_ATT", "3PT_PCT", "FT_ATT", "FT_PCT", "OFF_RB", "DEF_RB", "AVG_AST", "AVG_TO", "AVG_STL", "AVG_BLK", "EFG_PCT", "TS_PCT"]

m_team0_metric_dict = {d : dict() for d in metriclist}
m_team1_metric_dict = {d : dict() for d in metriclist}
m_team0_opp_metric_dict = {d : dict() for d in metriclist}
m_team1_opp_metric_dict = {d : dict() for d in metriclist}

for key in metriclist:
    m_team0_metric_dict[key] = {l : list() for l in yearlist}
    m_team1_metric_dict[key] = {l : list() for l in yearlist}
    m_team0_opp_metric_dict[key] = {l : list() for l in yearlist}
    m_team1_opp_metric_dict[key] = {l : list() for l in yearlist}

In [8]:
# Metric Creation for Team 0

for year in yearlist:
    year_results = m_reg_results_df.where(m_reg_results_df["Season"] == year).dropna()
    for team in m_team0_dict[year]:

        num_games_played = len(year_results.where((year_results["WTeamID"] == team) | (year_results["LTeamID"] == team)).dropna())

        # Avg. PPG
        totalpoints = year_results["WScore"].where(year_results["WTeamID"] == team).dropna().sum() + year_results["LScore"].where(year_results["LTeamID"] == team).dropna().sum()
        avgppg = round(totalpoints / num_games_played, 2)
        m_team0_metric_dict["AVG_PPG"][year].append(avgppg)

        # FG Att. and Pct.
        totalfgatt = year_results["WFGA"].where(year_results["WTeamID"] == team).dropna().sum() + year_results["LFGA"].where(year_results["LTeamID"] == team).dropna().sum()
        avgfgatt = round(totalfgatt / num_games_played, 2)
        totalfgmade = year_results["WFGM"].where(year_results["WTeamID"] == team).dropna().sum() + year_results["LFGM"].where(year_results["LTeamID"] == team).dropna().sum()
        fgpct = round(((totalfgmade / totalfgatt)*100), 2)
        m_team0_metric_dict["FG_ATT"][year].append(avgfgatt)
        m_team0_metric_dict["FG_PCT"][year].append(fgpct)

        # 3PT Att. and Pct.
        total3ptatt = year_results["WFGA3"].where(year_results["WTeamID"] == team).dropna().sum() + year_results["LFGA3"].where(year_results["LTeamID"] == team).dropna().sum()
        avg3ptatt = round(total3ptatt / num_games_played, 2)
        total3ptmade = year_results["WFGM3"].where(year_results["WTeamID"] == team).dropna().sum() + year_results["LFGM3"].where(year_results["LTeamID"] == team).dropna().sum()
        threeptpct = round(((total3ptmade / total3ptatt)*100), 2)
        m_team0_metric_dict["3PT_ATT"][year].append(avg3ptatt)
        m_team0_metric_dict["3PT_PCT"][year].append(threeptpct)

        # FT Att. and Pct.
        totalftatt = year_results["WFTA"].where(year_results["WTeamID"] == team).dropna().sum() + year_results["LFTA"].where(year_results["LTeamID"] == team).dropna().sum()
        avgftatt = round(totalftatt / num_games_played, 2)
        totalftmade = year_results["WFTM"].where(year_results["WTeamID"] == team).dropna().sum() + year_results["LFTM"].where(year_results["LTeamID"] == team).dropna().sum()
        ftpct = round(((totalftmade / totalftatt)*100), 2)
        m_team0_metric_dict["FT_ATT"][year].append(avgftatt)
        m_team0_metric_dict["FT_PCT"][year].append(ftpct)

        # Avg. Offensive and Defensive Rebounds
        totaloffrb = year_results["WOR"].where(year_results["WTeamID"] == team).dropna().sum() + year_results["LOR"].where(year_results["LTeamID"] == team).dropna().sum()
        avgoffrb = round(totaloffrb / num_games_played, 2)
        totaldefrb = year_results["WDR"].where(year_results["WTeamID"] == team).dropna().sum() + year_results["LDR"].where(year_results["LTeamID"] == team).dropna().sum()
        avgdefrb = round(totaldefrb / num_games_played, 2)
        m_team0_metric_dict["OFF_RB"][year].append(avgoffrb)
        m_team0_metric_dict["DEF_RB"][year].append(avgdefrb)

        # Avg. Assists
        totalassists = year_results["WAst"].where(year_results["WTeamID"] == team).dropna().sum() + year_results["LAst"].where(year_results["LTeamID"] == team).dropna().sum()
        avgassists = round(totalassists / num_games_played, 2)
        m_team0_metric_dict["AVG_AST"][year].append(avgassists)

        # Avg. Turnovers
        totalturnovers = year_results["WTO"].where(year_results["WTeamID"] == team).dropna().sum() + year_results["LTO"].where(year_results["LTeamID"] == team).dropna().sum()
        avgturnovers = round(totalturnovers / num_games_played, 2)
        m_team0_metric_dict["AVG_TO"][year].append(avgturnovers)

        # Avg. Steals
        totalsteals = year_results["WStl"].where(year_results["WTeamID"] == team).dropna().sum() + year_results["LStl"].where(year_results["LTeamID"] == team).dropna().sum()
        avgsteals = round(totalsteals / num_games_played, 2)
        m_team0_metric_dict["AVG_STL"][year].append(avgsteals)

        # Avg. Blocks
        totalblocks = year_results["WBlk"].where(year_results["WTeamID"] == team).dropna().sum() + year_results["LBlk"].where(year_results["LTeamID"] == team).dropna().sum()
        avgblocks = round(totalblocks / num_games_played, 2)
        m_team0_metric_dict["AVG_BLK"][year].append(avgblocks)

        # Effective Field Goal Percentage
        total2ptmade = totalfgmade - total3ptmade
        efg_pct = round(((total2ptmade + (1.5*total3ptmade)) / totalfgatt)*100, 2)
        m_team0_metric_dict["EFG_PCT"][year].append(efg_pct)

        # True Shooting Percentage
        ts_pct = round((totalpoints / (2*(totalfgatt + (0.44*totalftatt))))*100, 2)
        m_team0_metric_dict["TS_PCT"][year].append(ts_pct)

In [9]:
# Metric Creation for Team 1

for key in m_team0_metric_dict.keys():
    for year in yearlist:
        for i in range(0, len(m_team0_metric_dict[key][year])):
            if i%2 == 0:
                m_team1_metric_dict[key][year].append(m_team0_metric_dict[key][year][i])
            else:
                m_team1_metric_dict[key][year].insert(i-1, m_team0_metric_dict[key][year][i])

In [10]:
# Metric Creation for Team 0 (Opponent/Defensive Metric)

for year in yearlist:
    year_results = m_reg_results_df.where(m_reg_results_df["Season"] == year).dropna()
    for team in m_team0_dict[year]:

        num_games_played = len(year_results.where((year_results["WTeamID"] == team) | (year_results["LTeamID"] == team)).dropna())

        # Avg. PPG
        totalpoints = year_results["LScore"].where(year_results["WTeamID"] == team).dropna().sum() + year_results["WScore"].where(year_results["LTeamID"] == team).dropna().sum()
        avgppg = round(totalpoints / num_games_played, 2)
        m_team0_opp_metric_dict["AVG_PPG"][year].append(avgppg)

        # FG Att. and Pct.
        totalfgatt = year_results["LFGA"].where(year_results["WTeamID"] == team).dropna().sum() + year_results["WFGA"].where(year_results["LTeamID"] == team).dropna().sum()
        avgfgatt = round(totalfgatt / num_games_played, 2)
        totalfgmade = year_results["LFGM"].where(year_results["WTeamID"] == team).dropna().sum() + year_results["WFGM"].where(year_results["LTeamID"] == team).dropna().sum()
        fgpct = round(((totalfgmade / totalfgatt)*100), 2)
        m_team0_opp_metric_dict["FG_ATT"][year].append(avgfgatt)
        m_team0_opp_metric_dict["FG_PCT"][year].append(fgpct)

        # 3PT Att. and Pct.
        total3ptatt = year_results["LFGA3"].where(year_results["WTeamID"] == team).dropna().sum() + year_results["WFGA3"].where(year_results["LTeamID"] == team).dropna().sum()
        avg3ptatt = round(total3ptatt / num_games_played, 2)
        total3ptmade = year_results["LFGM3"].where(year_results["WTeamID"] == team).dropna().sum() + year_results["WFGM3"].where(year_results["LTeamID"] == team).dropna().sum()
        threeptpct = round(((total3ptmade / total3ptatt)*100), 2)
        m_team0_opp_metric_dict["3PT_ATT"][year].append(avg3ptatt)
        m_team0_opp_metric_dict["3PT_PCT"][year].append(threeptpct)

        # FT Att. and Pct.
        totalftatt = year_results["LFTA"].where(year_results["WTeamID"] == team).dropna().sum() + year_results["WFTA"].where(year_results["LTeamID"] == team).dropna().sum()
        avgftatt = round(totalftatt / num_games_played, 2)
        totalftmade = year_results["LFTM"].where(year_results["WTeamID"] == team).dropna().sum() + year_results["WFTM"].where(year_results["LTeamID"] == team).dropna().sum()
        ftpct = round(((totalftmade / totalftatt)*100), 2)
        m_team0_opp_metric_dict["FT_ATT"][year].append(avgftatt)
        m_team0_opp_metric_dict["FT_PCT"][year].append(ftpct)

        # Avg. Offensive and Defensive Rebounds
        totaloffrb = year_results["LOR"].where(year_results["WTeamID"] == team).dropna().sum() + year_results["WOR"].where(year_results["LTeamID"] == team).dropna().sum()
        avgoffrb = round(totaloffrb / num_games_played, 2)
        totaldefrb = year_results["LDR"].where(year_results["WTeamID"] == team).dropna().sum() + year_results["WDR"].where(year_results["LTeamID"] == team).dropna().sum()
        avgdefrb = round(totaldefrb / num_games_played, 2)
        m_team0_opp_metric_dict["OFF_RB"][year].append(avgoffrb)
        m_team0_opp_metric_dict["DEF_RB"][year].append(avgdefrb)

        # Avg. Assists
        totalassists = year_results["LAst"].where(year_results["WTeamID"] == team).dropna().sum() + year_results["WAst"].where(year_results["LTeamID"] == team).dropna().sum()
        avgassists = round(totalassists / num_games_played, 2)
        m_team0_opp_metric_dict["AVG_AST"][year].append(avgassists)

        # Avg. Turnovers
        totalturnovers = year_results["LTO"].where(year_results["WTeamID"] == team).dropna().sum() + year_results["WTO"].where(year_results["LTeamID"] == team).dropna().sum()
        avgturnovers = round(totalturnovers / num_games_played, 2)
        m_team0_opp_metric_dict["AVG_TO"][year].append(avgturnovers)

        # Avg. Steals
        totalsteals = year_results["LStl"].where(year_results["WTeamID"] == team).dropna().sum() + year_results["WStl"].where(year_results["LTeamID"] == team).dropna().sum()
        avgsteals = round(totalsteals / num_games_played, 2)
        m_team0_opp_metric_dict["AVG_STL"][year].append(avgsteals)

        # Avg. Blocks
        totalblocks = year_results["LBlk"].where(year_results["WTeamID"] == team).dropna().sum() + year_results["WBlk"].where(year_results["LTeamID"] == team).dropna().sum()
        avgblocks = round(totalblocks / num_games_played, 2)
        m_team0_opp_metric_dict["AVG_BLK"][year].append(avgblocks)

        # Effective Field Goal Percentage
        total2ptmade = totalfgmade - total3ptmade
        efg_pct = round(((total2ptmade + (1.5*total3ptmade)) / totalfgatt)*100, 2)
        m_team0_opp_metric_dict["EFG_PCT"][year].append(efg_pct)

        # True Shooting Percentage
        ts_pct = round((totalpoints / (2*(totalfgatt + (0.44*totalftatt))))*100, 2)
        m_team0_opp_metric_dict["TS_PCT"][year].append(ts_pct)

In [11]:
# Metric Creation for Team 1 (Opponent/Defensive Metric)

for key in m_team0_opp_metric_dict.keys():
    for year in yearlist:
        for i in range(0, len(m_team0_opp_metric_dict[key][year])):
            if i%2 == 0:
                m_team1_opp_metric_dict[key][year].append(m_team0_opp_metric_dict[key][year][i])
            else:
                m_team1_opp_metric_dict[key][year].insert(i-1, m_team0_opp_metric_dict[key][year][i])

In [12]:
# Combining all metric lists and creating columns for full dataset

m_trainset_team0_cols = {l : list() for l in metriclist}
m_trainset_team1_cols = {l : list() for l in metriclist}
m_trainset_team0_opp_cols = {l : list() for l in metriclist}
m_trainset_team1_opp_cols = {l : list() for l in metriclist}

for key in metriclist:
    m_trainset_team0_cols[key] = []
    m_trainset_team1_cols[key] = []
    m_trainset_team0_opp_cols[key] = []
    m_trainset_team1_opp_cols[key] = []

    for year in yearlist:
        if year == 2020 or year == 2024:
            continue
        m_trainset_team0_cols[key] += m_team0_metric_dict[key][year]
        m_trainset_team1_cols[key] += m_team1_metric_dict[key][year]
        m_trainset_team0_opp_cols[key] += m_team0_opp_metric_dict[key][year]
        m_trainset_team1_opp_cols[key] += m_team1_opp_metric_dict[key][year]

In [13]:
# Inserting all metric columns into trainset

# Team 0
for key in metriclist:
    m_trainset_full.insert(len(m_trainset_full.columns), f"T0 {key}", m_trainset_team0_cols[key])

# Team 1
for key in metriclist:
    m_trainset_full.insert(len(m_trainset_full.columns), f"T1 {key}", m_trainset_team1_cols[key])

# Team 0 (Opponent Metric)
for key in metriclist:
    m_trainset_full.insert(len(m_trainset_full.columns), f"T0 Opp. {key}", m_trainset_team0_opp_cols[key])

# Team 1 (Opponent Metric)
for key in metriclist:
    m_trainset_full.insert(len(m_trainset_full.columns), f"T1 Opp. {key}", m_trainset_team1_opp_cols[key])

In [14]:
# Inserting all metric columns into predset

# Team 0
for key in metriclist:
    m_predset_full.insert(len(m_predset_full.columns), f"T0 {key}", m_team0_metric_dict[key][2024])

# Team 1
for key in metriclist:
    m_predset_full.insert(len(m_predset_full.columns), f"T1 {key}", m_team1_metric_dict[key][2024])

# Team 0 (Opponent Metric)
for key in metriclist:
    m_predset_full.insert(len(m_predset_full.columns), f"T0 Opp. {key}", m_team0_opp_metric_dict[key][2024])

# Team 1 (Opponent Metric)
for key in metriclist:
    m_predset_full.insert(len(m_predset_full.columns), f"T1 Opp. {key}", m_team1_opp_metric_dict[key][2024])

<div id="modeling"></a>

### Preparing and Running the Model

In [15]:
# Preparing trainset for modeling

m_trainset_formodel = m_trainset_full.drop(columns = ["Season", "Team0", "Team1"])
scaler = MinMaxScaler()
m_trainset_formodel_scaled = m_trainset_formodel.drop("Score", axis = 1)
m_trainset_formodel_scaled[m_trainset_formodel_scaled.columns] = scaler.fit_transform(m_trainset_formodel_scaled[m_trainset_formodel_scaled.columns])
X, y = m_trainset_formodel_scaled, m_trainset_formodel[["Score"]]
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 1)

dtrain_reg = xgb.DMatrix(X_train, y_train)
dtest_reg = xgb.DMatrix(X_test, y_test)

In [16]:
# Running the model and model evaluation

params = {"objective": "reg:squarederror", "tree_method": "auto"}
n = 1000
evals = [(dtrain_reg , "train"), (dtest_reg, "validation")]

model = xgb.train(
    params = params,
    dtrain = dtrain_reg,
    num_boost_round = n,
    evals = evals,
    verbose_eval = 10,
    early_stopping_rounds = 20
)

preds = model.predict(dtest_reg)

rmse = mean_squared_error(y_test, preds, squared = False)
print(f"RMSE of the base model: {rmse:.3f}")

[0]	train-rmse:49.69340	validation-rmse:49.46804


[10]	train-rmse:7.28689	validation-rmse:10.89699
[20]	train-rmse:5.47924	validation-rmse:10.91335
[30]	train-rmse:4.41005	validation-rmse:10.89943
[35]	train-rmse:3.89647	validation-rmse:10.90875
RMSE of the base model: 10.909


In [17]:
# Using the model on prediction data

m_predset_formodel = m_predset_formodel_scaled = m_predset_full.drop(columns = ["Season", "Seed", "Team0", "Team1"])
m_predset_formodel_scaled[m_predset_formodel_scaled.columns] = scaler.fit_transform(m_predset_formodel[m_predset_formodel.columns])
m_predset_matrix = xgb.DMatrix(m_predset_formodel_scaled)
prediction = model.predict(m_predset_matrix)

high_seed_prediction = prediction[::2]
low_seed_prediction = prediction[1::2]
round1_results_df = pd.DataFrame()
round1_results_df.insert(0, "High Seed Score", high_seed_prediction)
round1_results_df.insert(1, "Low Seed Score", low_seed_prediction)

round1_results_df.head()

,High Seed Score,Low Seed Score
0,74.936470,58.515148
1,73.760429,54.025265
2,69.370857,60.200806
3,73.249214,67.577072
4,65.380829,65.950073


<div id="predicting"></a>

### Predicting Every Round

In [18]:
# Function to create bracket of seeds out of a list of predicted scores.

def bracketizer(pred_scores, current_seeds_round: list[str], round: int, is_bracket: bool) -> list[str]:
    new_seeds_list = []

    for i in range(0, len(pred_scores)):
        if i%2 != 0:
            continue
        if pred_scores[i] > pred_scores[i+1]:
            new_seeds_list.append(current_seeds_round[i])
        else:
            new_seeds_list.append(current_seeds_round[i+1])
    
    if round > 3 or not is_bracket:
        return new_seeds_list

    region = ["W", "X", "Y", "Z"]
    new_seeds_bracket = []
    for r in region:
        region_seeds = [s for s in new_seeds_list if r in s]
        l = len(region_seeds)
        bracket = [None]*l
        bracket[::2] = region_seeds[:int(l/2)]
        bracket[1::2] = reversed(region_seeds[int(l/2):])
        new_seeds_bracket += bracket

    return new_seeds_bracket

In [19]:
# Function to produce columns of metrics for each round's subsequent df

def metricizer(bracket: list, category: str) -> dict:
    predset_metrics_list = {l: list() for l in metriclist}
    if "Opp" not in category:
        metric_dict = m_team0_metric_dict
    else:
        metric_dict = m_team0_opp_metric_dict

    for metric in metriclist:
        predset_metrics_perteam = {f : float() for f in bracket}
        for i in range(len(m_current_seeds_bracket_team0)):
            predset_metrics_perteam[m_current_seeds_bracket_team0[i]] = metric_dict[metric][2024][i]
        
        for team in bracket:
            predset_metrics_list[metric].append(predset_metrics_perteam[team])
        
    return predset_metrics_list

In [20]:
# Determining winners/losers and creating the bracket for the subsequent round

m_current_seeds_bracket_round2 = bracketizer(prediction, m_current_seeds_bracket, 2, True)

In [21]:
# Creating predset for next round

m_predset_round2 = pd.DataFrame({"Seed":m_current_seeds_bracket_round2})

# Creating column lists for team 0 and team 1
m_current_seeds_bracket_round2_team0 = [m_current_seeds_teamdict[seed] for seed in m_current_seeds_bracket_round2]
m_current_seeds_bracket_round2_team1 = [0]*len(m_current_seeds_bracket_round2_team0)
m_current_seeds_bracket_round2_team1[::2] = m_current_seeds_bracket_round2_team0[1::2]
m_current_seeds_bracket_round2_team1[1::2] = m_current_seeds_bracket_round2_team0[::2]

# Creating column lists with metrics for all round 2 teams
m_predset_metrics_round2_list_team0 = metricizer(m_current_seeds_bracket_round2_team0, "T0")
m_predset_metrics_round2_list_team1 = metricizer(m_current_seeds_bracket_round2_team1, "T1")
m_predset_opp_metrics_round2_list_team0 = metricizer(m_current_seeds_bracket_round2_team0, "T0 Opp")
m_predset_opp_metrics_round2_list_team1 = metricizer(m_current_seeds_bracket_round2_team1, "T1 Opp")

# Inserting column lists into new dataframe
m_predset_round2.insert(len(m_predset_round2.columns), "Team0", m_current_seeds_bracket_round2_team0)
m_predset_round2.insert(len(m_predset_round2.columns), "Team1", m_current_seeds_bracket_round2_team1)

for metric in metriclist:
    m_predset_round2.insert(len(m_predset_round2.columns), f"T0 {metric}", m_predset_metrics_round2_list_team0[metric])

for metric in metriclist:
    m_predset_round2.insert(len(m_predset_round2.columns), f"T1 {metric}", m_predset_metrics_round2_list_team1[metric])

for metric in metriclist:
    m_predset_round2.insert(len(m_predset_round2.columns), f"T0 Opp. {metric}", m_predset_opp_metrics_round2_list_team0[metric])

for metric in metriclist:
    m_predset_round2.insert(len(m_predset_round2.columns), f"T1 Opp. {metric}", m_predset_opp_metrics_round2_list_team1[metric])

In [22]:
# Using the model on round 2 prediction data

m_predset_round2_formodel = m_predset_round2_formodel_scaled = m_predset_round2.drop(columns = ["Seed", "Team0", "Team1"])
m_predset_round2_formodel_scaled[m_predset_round2_formodel_scaled.columns] = scaler.fit_transform(m_predset_round2_formodel[m_predset_round2_formodel.columns])
m_predset_round2_matrix = xgb.DMatrix(m_predset_round2_formodel_scaled)
round2_prediction = model.predict(m_predset_round2_matrix)

high_seed_round2_prediction = round2_prediction[::2]
low_seed_round2_prediction = round2_prediction[1::2]
round2_results_df = pd.DataFrame()
round2_results_df.insert(0, "High Seed Score", high_seed_round2_prediction)
round2_results_df.insert(1, "Low Seed Score", low_seed_round2_prediction)

round2_results_df.head()

,High Seed Score,Low Seed Score
0,73.686646,53.033836
1,59.297211,58.041889
2,79.275978,71.836800
3,78.949348,62.875053
4,66.851837,49.190331


In [23]:
# Rinsing and repeating for every round, changing small aspects to accommodate different bracket sizes
# ROUND 3

# Determining winners/losers and creating the bracket for the subsequent round

m_current_seeds_bracket_round3 = bracketizer(round2_prediction, m_current_seeds_bracket_round2, 3, True)

# Creating predset for next round

m_predset_round3 = pd.DataFrame()
m_predset_round3.insert(0, "Seed", m_current_seeds_bracket_round3)

# Creating column lists for team 0 and team 1
m_current_seeds_bracket_round3_team0 = [m_current_seeds_teamdict[seed] for seed in m_current_seeds_bracket_round3]
m_current_seeds_bracket_round3_team1 = [0]*len(m_current_seeds_bracket_round3_team0)
m_current_seeds_bracket_round3_team1[::2] = m_current_seeds_bracket_round3_team0[1::2]
m_current_seeds_bracket_round3_team1[1::2] = m_current_seeds_bracket_round3_team0[::2]

# Creating column lists with metrics for all round 3 teams
m_predset_metrics_round3_list_team0 = metricizer(m_current_seeds_bracket_round3_team0, "T0")
m_predset_metrics_round3_list_team1 = metricizer(m_current_seeds_bracket_round3_team1, "T1")
m_predset_opp_metrics_round3_list_team0 = metricizer(m_current_seeds_bracket_round3_team0, "T0 Opp")
m_predset_opp_metrics_round3_list_team1 = metricizer(m_current_seeds_bracket_round3_team1, "T1 Opp")

# Inserting column lists into new dataframe
m_predset_round3.insert(len(m_predset_round3.columns), "Team0", m_current_seeds_bracket_round3_team0)
m_predset_round3.insert(len(m_predset_round3.columns), "Team1", m_current_seeds_bracket_round3_team1)

for metric in metriclist:
    m_predset_round3.insert(len(m_predset_round3.columns), f"T0 {metric}", m_predset_metrics_round3_list_team0[metric])

for metric in metriclist:
    m_predset_round3.insert(len(m_predset_round3.columns), f"T1 {metric}", m_predset_metrics_round3_list_team1[metric])

for metric in metriclist:
    m_predset_round3.insert(len(m_predset_round3.columns), f"T0 Opp. {metric}", m_predset_opp_metrics_round3_list_team0[metric])

for metric in metriclist:
    m_predset_round3.insert(len(m_predset_round3.columns), f"T1 Opp. {metric}", m_predset_opp_metrics_round3_list_team1[metric])

In [24]:
# Using the model on round 3 prediction data

m_predset_round3_formodel = m_predset_round3_formodel_scaled = m_predset_round3.drop(columns = ["Seed", "Team0", "Team1"])
m_predset_round3_formodel_scaled[m_predset_round3_formodel_scaled.columns] = scaler.fit_transform(m_predset_round3_formodel[m_predset_round3_formodel.columns])
m_predset_round3_matrix = xgb.DMatrix(m_predset_round3_formodel_scaled)
round3_prediction = model.predict(m_predset_round3_matrix)

high_seed_round3_prediction = round3_prediction[::2]
low_seed_round3_prediction = round3_prediction[1::2]
round3_results_df = pd.DataFrame()
round3_results_df.insert(0, "High Seed Score", high_seed_round3_prediction)
round3_results_df.insert(1, "Low Seed Score", low_seed_round3_prediction)

round3_results_df.head()

,High Seed Score,Low Seed Score
0,75.331779,73.453682
1,59.561405,69.684303
2,48.639175,71.934647
3,55.140636,70.399124
4,82.057396,79.821419


In [25]:
# Rinsing and repeating for every round, changing small aspects to accommodate different bracket sizes
# ROUND 4


# Determining winners/losers and creating the bracket for the subsequent round

m_current_seeds_bracket_round4 = bracketizer(round3_prediction, m_current_seeds_bracket_round3, 4, True)

# Creating predset for next round

m_predset_round4 = pd.DataFrame()
m_predset_round4.insert(0, "Seed", m_current_seeds_bracket_round4)

# Creating column lists for team 0 and team 1
m_current_seeds_bracket_round4_team0 = [m_current_seeds_teamdict[seed] for seed in m_current_seeds_bracket_round4]
m_current_seeds_bracket_round4_team1 = [0]*len(m_current_seeds_bracket_round4_team0)
m_current_seeds_bracket_round4_team1[::2] = m_current_seeds_bracket_round4_team0[1::2]
m_current_seeds_bracket_round4_team1[1::2] = m_current_seeds_bracket_round4_team0[::2]

# Creating column lists with metrics for all round 4 teams
m_predset_metrics_round4_list_team0 = metricizer(m_current_seeds_bracket_round4_team0, "T0")
m_predset_metrics_round4_list_team1 = metricizer(m_current_seeds_bracket_round4_team1, "T1")
m_predset_opp_metrics_round4_list_team0 = metricizer(m_current_seeds_bracket_round4_team0, "T0 Opp")
m_predset_opp_metrics_round4_list_team1 = metricizer(m_current_seeds_bracket_round4_team1, "T1 Opp")

# Inserting column lists into new dataframe
m_predset_round4.insert(len(m_predset_round4.columns), "Team0", m_current_seeds_bracket_round4_team0)
m_predset_round4.insert(len(m_predset_round4.columns), "Team1", m_current_seeds_bracket_round4_team1)

for metric in metriclist:
    m_predset_round4.insert(len(m_predset_round4.columns), f"T0 {metric}", m_predset_metrics_round4_list_team0[metric])

for metric in metriclist:
    m_predset_round4.insert(len(m_predset_round4.columns), f"T1 {metric}", m_predset_metrics_round4_list_team1[metric])

for metric in metriclist:
    m_predset_round4.insert(len(m_predset_round4.columns), f"T0 Opp. {metric}", m_predset_opp_metrics_round4_list_team0[metric])

for metric in metriclist:
    m_predset_round4.insert(len(m_predset_round4.columns), f"T1 Opp. {metric}", m_predset_opp_metrics_round4_list_team1[metric])

In [26]:
# Using the model on round 4 prediction data

m_predset_round4_formodel = m_predset_round4_formodel_scaled = m_predset_round4.drop(columns = ["Seed", "Team0", "Team1"])
m_predset_round4_formodel_scaled[m_predset_round4_formodel_scaled.columns] = scaler.fit_transform(m_predset_round4_formodel[m_predset_round4_formodel.columns])
m_predset_round4_matrix = xgb.DMatrix(m_predset_round4_formodel_scaled)
round4_prediction = model.predict(m_predset_round4_matrix)

high_seed_round4_prediction = round4_prediction[::2]
low_seed_round4_prediction = round4_prediction[1::2]
round4_results_df = pd.DataFrame()
round4_results_df.insert(0, "High Seed Score", high_seed_round4_prediction)
round4_results_df.insert(1, "Low Seed Score", low_seed_round4_prediction)

round4_results_df.head()

,High Seed Score,Low Seed Score
0,75.017860,65.179695
1,66.063499,55.763836
2,72.934456,87.569809
3,73.090408,52.888306


In [27]:
# Rinsing and repeating for every round, changing small aspects to accommodate different bracket sizes
# ROUND 5


# Determining winners/losers and creating the bracket for the subsequent round

m_current_seeds_bracket_round5 = bracketizer(round4_prediction, m_current_seeds_bracket_round4, 5, True)

# Creating predset for next round

m_predset_round5 = pd.DataFrame()
m_predset_round5.insert(0, "Seed", m_current_seeds_bracket_round5)

# Creating column lists for team 0 and team 1
m_current_seeds_bracket_round5_team0 = [m_current_seeds_teamdict[seed] for seed in m_current_seeds_bracket_round5]
m_current_seeds_bracket_round5_team1 = [0]*len(m_current_seeds_bracket_round5_team0)
m_current_seeds_bracket_round5_team1[::2] = m_current_seeds_bracket_round5_team0[1::2]
m_current_seeds_bracket_round5_team1[1::2] = m_current_seeds_bracket_round5_team0[::2]

# Creating column lists with metrics for all round 5 teams
m_predset_metrics_round5_list_team0 = metricizer(m_current_seeds_bracket_round5_team0, "T0")
m_predset_metrics_round5_list_team1 = metricizer(m_current_seeds_bracket_round5_team1, "T1")
m_predset_opp_metrics_round5_list_team0 = metricizer(m_current_seeds_bracket_round5_team0, "T0 Opp")
m_predset_opp_metrics_round5_list_team1 = metricizer(m_current_seeds_bracket_round5_team1, "T1 Opp")

# Inserting column lists into new dataframe
m_predset_round5.insert(len(m_predset_round5.columns), "Team0", m_current_seeds_bracket_round5_team0)
m_predset_round5.insert(len(m_predset_round5.columns), "Team1", m_current_seeds_bracket_round5_team1)

for metric in metriclist:
    m_predset_round5.insert(len(m_predset_round5.columns), f"T0 {metric}", m_predset_metrics_round5_list_team0[metric])

for metric in metriclist:
    m_predset_round5.insert(len(m_predset_round5.columns), f"T1 {metric}", m_predset_metrics_round5_list_team1[metric])

for metric in metriclist:
    m_predset_round5.insert(len(m_predset_round5.columns), f"T0 Opp. {metric}", m_predset_opp_metrics_round5_list_team0[metric])

for metric in metriclist:
    m_predset_round5.insert(len(m_predset_round5.columns), f"T1 Opp. {metric}", m_predset_opp_metrics_round5_list_team1[metric])

In [28]:
# Using the model on round 5 prediction data

m_predset_round5_formodel = m_predset_round5_formodel_scaled = m_predset_round5.drop(columns = ["Seed", "Team0", "Team1"])
m_predset_round5_formodel_scaled[m_predset_round5_formodel_scaled.columns] = scaler.fit_transform(m_predset_round5_formodel[m_predset_round5_formodel.columns])
m_predset_round5_matrix = xgb.DMatrix(m_predset_round5_formodel_scaled)
round5_prediction = model.predict(m_predset_round5_matrix)

high_seed_round5_prediction = round5_prediction[::2]
low_seed_round5_prediction = round5_prediction[1::2]
round5_results_df = pd.DataFrame()
round5_results_df.insert(0, "High Seed Score", high_seed_round5_prediction)
round5_results_df.insert(1, "Low Seed Score", low_seed_round5_prediction)

round5_results_df.head()

,High Seed Score,Low Seed Score
0,62.158661,37.638378
1,70.510391,51.665657


In [29]:
# Rinsing and repeating for every round, changing small aspects to accommodate different bracket sizes
# ROUND 6


# Determining winners/losers and creating the bracket for the subsequent round

m_current_seeds_bracket_round6 = bracketizer(round5_prediction, m_current_seeds_bracket_round5, 6, True)

# Creating predset for next round

m_predset_round6 = pd.DataFrame()
m_predset_round6.insert(0, "Seed", m_current_seeds_bracket_round6)

# Creating column lists for team 0 and team 1
m_current_seeds_bracket_round6_team0 = [m_current_seeds_teamdict[seed] for seed in m_current_seeds_bracket_round6]
m_current_seeds_bracket_round6_team1 = [0]*len(m_current_seeds_bracket_round6_team0)
m_current_seeds_bracket_round6_team1[::2] = m_current_seeds_bracket_round6_team0[1::2]
m_current_seeds_bracket_round6_team1[1::2] = m_current_seeds_bracket_round6_team0[::2]


# Creating column lists with metrics for all round 6 teams
m_predset_metrics_round6_list_team0 = metricizer(m_current_seeds_bracket_round6_team0, "T0")
m_predset_metrics_round6_list_team1 = metricizer(m_current_seeds_bracket_round6_team1, "T1")
m_predset_opp_metrics_round6_list_team0 = metricizer(m_current_seeds_bracket_round6_team0, "T0 Opp")
m_predset_opp_metrics_round6_list_team1 = metricizer(m_current_seeds_bracket_round6_team1, "T1 Opp")

# Inserting column lists into new dataframe
m_predset_round6.insert(len(m_predset_round6.columns), "Team0", m_current_seeds_bracket_round6_team0)
m_predset_round6.insert(len(m_predset_round6.columns), "Team1", m_current_seeds_bracket_round6_team1)

for metric in metriclist:
    m_predset_round6.insert(len(m_predset_round6.columns), f"T0 {metric}", m_predset_metrics_round6_list_team0[metric])

for metric in metriclist:
    m_predset_round6.insert(len(m_predset_round6.columns), f"T1 {metric}", m_predset_metrics_round6_list_team1[metric])

for metric in metriclist:
    m_predset_round6.insert(len(m_predset_round6.columns), f"T0 Opp. {metric}", m_predset_opp_metrics_round6_list_team0[metric])

for metric in metriclist:
    m_predset_round6.insert(len(m_predset_round6.columns), f"T1 Opp. {metric}", m_predset_opp_metrics_round6_list_team1[metric])

In [30]:
# Using the model on round 6 prediction data

m_predset_round6_formodel = m_predset_round6_formodel_scaled = m_predset_round6.drop(columns = ["Seed", "Team0", "Team1"])
m_predset_round6_formodel_scaled[m_predset_round6_formodel_scaled.columns] = scaler.fit_transform(m_predset_round6_formodel[m_predset_round6_formodel.columns])
m_predset_round6_matrix = xgb.DMatrix(m_predset_round6_formodel_scaled)
round6_prediction = model.predict(m_predset_round6_matrix)

high_seed_round6_prediction = round6_prediction[::2]
low_seed_round6_prediction = round6_prediction[1::2]
round6_results_df = pd.DataFrame()
round6_results_df.insert(0, "High Seed Score", high_seed_round6_prediction)
round6_results_df.insert(1, "Low Seed Score", low_seed_round6_prediction)

round6_results_df.head()

,High Seed Score,Low Seed Score
0,66.598991,36.523792


In [31]:
# Calculating the final winner

m_current_seeds_winner = bracketizer(round6_prediction, m_current_seeds_bracket_round6, 7, True)
m_current_seeds_winner

['W01']

In [34]:
# Overall Bracket Table

m_current_seeds_bracket_ovr = m_current_seeds_bracket + m_current_seeds_bracket_round2 + m_current_seeds_bracket_round3 + m_current_seeds_bracket_round4 + m_current_seeds_bracket_round5 + m_current_seeds_bracket_round6 + m_current_seeds_winner
m_current_seeds_bracket_ovr_df = pd.DataFrame()

# Seed

m_current_seeds_bracket_ovr_col_team0 = []
m_current_seeds_bracket_ovr_col_team1 = []
m_current_seeds_bracket_ovr_col_winner = []

m_current_seeds_bracket_ovr_col_team0 += m_current_seeds_bracket[::2] + m_current_seeds_bracket_round2[::2] + m_current_seeds_bracket_round3[::2] + m_current_seeds_bracket_round4[::2] + m_current_seeds_bracket_round5[::2] + m_current_seeds_bracket_round6[::2]
m_current_seeds_bracket_ovr_col_team1 += m_current_seeds_bracket[1::2] + m_current_seeds_bracket_round2[1::2] + m_current_seeds_bracket_round3[1::2] + m_current_seeds_bracket_round4[1::2] + m_current_seeds_bracket_round5[1::2] + m_current_seeds_bracket_round6[1::2]
m_current_seeds_bracket_ovr_col_winner += bracketizer(prediction, m_current_seeds_bracket, 2, False) + bracketizer(round2_prediction, m_current_seeds_bracket_round2, 3, False) + bracketizer(round3_prediction, m_current_seeds_bracket_round3, 4, False) + bracketizer(round4_prediction, m_current_seeds_bracket_round4, 5, False) + bracketizer(round5_prediction, m_current_seeds_bracket_round5, 6, False) + m_current_seeds_winner

m_current_seeds_bracket_ovr_df.insert(0, "Team 0", m_current_seeds_bracket_ovr_col_team0)
m_current_seeds_bracket_ovr_df.insert(1, "Team 1", m_current_seeds_bracket_ovr_col_team1)
m_current_seeds_bracket_ovr_df.insert(2, "Winner", m_current_seeds_bracket_ovr_col_winner)

# Team

m_teams_file = open("./Data/MTeams.csv")
m_teams_df = pd.read_csv(m_teams_file)

m_current_seeds_bracket_ovr_col_team0_teamid = []
m_current_seeds_bracket_ovr_col_team1_teamid = []
m_current_seeds_bracket_ovr_col_winner_teamid = []

for i in range(0, len(m_current_seeds_bracket_ovr_col_team0)):
    m_current_seeds_bracket_ovr_col_team0_teamid.append(m_current_seeds_teamdict[m_current_seeds_bracket_ovr_col_team0[i]])
    m_current_seeds_bracket_ovr_col_team1_teamid.append(m_current_seeds_teamdict[m_current_seeds_bracket_ovr_col_team1[i]])
    m_current_seeds_bracket_ovr_col_winner_teamid.append(m_current_seeds_teamdict[m_current_seeds_bracket_ovr_col_winner[i]])

m_teams_dict = {s : str() for s in m_teams_df["TeamID"]}
for teamid in m_teams_dict.keys():
    m_teams_dict[teamid] = m_teams_df["TeamName"].where(m_teams_df["TeamID"] == teamid).dropna().tolist()[0]

m_current_seeds_bracket_ovr_col_team0_teamname = []
m_current_seeds_bracket_ovr_col_team1_teamname = []
m_current_seeds_bracket_ovr_col_winner_teamname = []

for i in range(0, len(m_current_seeds_bracket_ovr_col_team0_teamid)):
    m_current_seeds_bracket_ovr_col_team0_teamname.append(m_teams_dict[m_current_seeds_bracket_ovr_col_team0_teamid[i]])
    m_current_seeds_bracket_ovr_col_team1_teamname.append(m_teams_dict[m_current_seeds_bracket_ovr_col_team1_teamid[i]])
    m_current_seeds_bracket_ovr_col_winner_teamname.append(m_teams_dict[m_current_seeds_bracket_ovr_col_winner_teamid[i]])

m_current_seeds_bracket_ovr_df.insert(1, "Team 0 Name", m_current_seeds_bracket_ovr_col_team0_teamname)
m_current_seeds_bracket_ovr_df.insert(3, "Team 1 Name", m_current_seeds_bracket_ovr_col_team1_teamname)
m_current_seeds_bracket_ovr_df.insert(5, "Winner Name", m_current_seeds_bracket_ovr_col_winner_teamname)

# Round Column

m_current_seeds_bracket_ovr_col_round = []
for i in range(0, 63):
    if i < 32:
        m_current_seeds_bracket_ovr_col_round.append("R1")
    elif (i >= 32 and i < 48):
        m_current_seeds_bracket_ovr_col_round.append("R2")
    elif (i >= 48 and i < 56):
        m_current_seeds_bracket_ovr_col_round.append("R3")
    elif (i >= 56 and i < 60):
        m_current_seeds_bracket_ovr_col_round.append("R4")
    elif (i >= 60 and i < 62):
        m_current_seeds_bracket_ovr_col_round.append("R5")
    elif i == 62:
        m_current_seeds_bracket_ovr_col_round.append("R6")

m_current_seeds_bracket_ovr_df.insert(0, "Round", m_current_seeds_bracket_ovr_col_round)

<div id="bracket"></a>

### The Final Bracket

In [35]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(m_current_seeds_bracket_ovr_df)

,Round,Team 0,Team 0 Name,Team 1,Team 1 Name,Winner,Winner Name
0,R1,W01,Connecticut,W16,Stetson,W01,Connecticut
1,R1,W02,Iowa St,W15,S Dakota St,W02,Iowa St
2,R1,W03,Illinois,W14,Morehead St,W03,Illinois
3,R1,W04,Auburn,W13,Yale,W04,Auburn
4,R1,W05,San Diego St,W12,UAB,W12,UAB
5,R1,W06,BYU,W11,Duquesne,W06,BYU
6,R1,W07,Washington St,W10,Drake,W07,Washington St
7,R1,W08,FL Atlantic,W09,Northwestern,W09,Northwestern
8,R1,X01,North Carolina,X16,Wagner,X01,North Carolina
9,R1,X02,Arizona,X15,Long Beach St,X02,Arizona
